In [7]:
import re

def read_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return text

def preprocess_text(text):
    """文本预处理：清洗、分句、去除无关字符"""
    # 分句
    sentences = re.split(r'(?<=[。！？\?])', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 5]
    
    # 清洗
    cleaned_sentences = []
    for sentence in sentences:
        # 去除引号、括号等字符
        cleaned = re.sub(r'[《》“”\n（）()]', '', sentence)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

In [8]:
text1 = read_text("text1.txt")
preprocess_text(text1)

['据外媒报道，美国两架海军军机26日分别坠毁在南海，无人员伤亡。',
 '第一起坠机事件涉及一架MH-60R海鹰直升机。',
 '根据美国海军太平洋舰队的声明，这架直升机在尼米兹号航空母舰进行例行操作时坠入南海。',
 '声明称，直升机上的三名机组人员被搜救队救起。',
 '半小时后，一架波音F/A-18F超级大黄蜂战斗机在尼米兹号航空母舰执行例行任务时也坠毁在南海。',
 '机上两名机组人员成功弹射逃生，被安全救起。',
 '据美国海军称，所有相关人员均安全且情况稳定。',
 '两起事故原因正在调查中。',
 ' 军事专家张军社27日接受环球时报采访时表示，美国在南海一天内先后坠毁一架舰载战斗机和一架直升机，这一事件并非偶然。',
 '美军长期在南海、亚太及全球范围内维持高强度战备状态，不断进行军事部署，以维持其霸权地位和国际警察角色。',
 '长期高压运作使美军兵力紧张、人员疲惫，事故发生的风险自然随之上升。',
 '他认为，此次事故很可能与操作疏忽或过度疲劳等因素有关。',
 '军事专家宋忠平27日接受环球时报采访时也持相似观点。',
 '他指出，美军长期以所谓航行自由为借口，在南海频繁炫耀武力，意在彰显其军事存在。',
 '表面上看，美国作为军事霸主仍在维持强势姿态，但实际上，即便拥有11 艘航空母舰，面对如此繁重的任务，美军也已力不从心。',
 '宋忠平分析称，美军航母肩负全球部署和多重任务，长期在中东及其他地区高强度执行作战和训练，加之部分官兵存在懈怠、厌战情绪，导致安全风险上升。',
 '因此，同一天发生两起坠机事故虽令人震惊，但并不令人意外。']

In [11]:
import spacy
import jieba
import jieba.posseg as pseg
import re
from collections import defaultdict

class DependencyTripleExtractor:
    def __init__(self):
        self.nlp = spacy.load("zh_core_web_sm")
        # 加载领域词典
        # self.load_domain_dictionary()
        
        # 初始化关系映射规则
        self.init_relation_relus()
    
    def init_relation_relus(self):
        """初始化依存关系到语义关系的映射规则"""
        # 基础依存关系映射
        self.dep_to_relation = {
            'nsubj': '',  # 主语，使用动词本身
            'dobj': '',   # 宾语，使用动词本身
            'nmod:poss': '的',  # 属格关系
            'prep': '',   # 介词，与动词组合
            'pobj': '',   # 介词宾语
            'appos': '是',  # 同位语关系
            'amod': '',   # 形容词修饰
            'nummod': ''  # 数量修饰
        }
    
    def preprocess_text(self, text):
        """文本预处理：清洗、分句、去除无关字符"""
        # 分句
        sentences = re.split(r'(?<=[。！？\?])', text)
        sentences = [s.strip() for s in sentences if len(s.strip()) > 5]
        
        # 清洗
        cleaned_sentences = []
        for sentence in sentences:
            # 去除引号、括号等字符
            cleaned = re.sub(r'[《》“”\n（）()]', '', sentence)
            cleaned_sentences.append(cleaned)
        return cleaned_sentences
    
    def analyze_sentence(self, sentence):
        """分析句子结构：分词、词性标注、依存句法分析"""
        # 实验spacy进行依存分析
        doc = self.nlp(sentence)
        
        # 构建依存关系图
        dependency_info = []
        root_token = None
        for token in doc:
            dep_info = {
                'text': token.text,
                'lemma': token.lemma_,
                'pos': token.pos_,
                'dep': token.dep_,
                'head_text': token.head.text,
                'head_pos': token.head.pos_,
                'is_root': token.dep == 'ROOT'
            }
            dependency_info.append(dep_info)
            if token.dep == 'ROOT':
                root_token = token
        return doc, dependency_info, root_token
    

ModuleNotFoundError: No module named 'spacy'